<a href="https://colab.research.google.com/github/cedro3/Transformer/blob/master/200725_7_7_transformer_training_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7.7 Transformerの学習・推論、判定根拠の可視化を実装

- 本ファイルでは、ここまでで作成したTransformerモデルとIMDbのDataLoaderを使用してクラス分類を学習させます。テストデータで推論をし、さらに判断根拠となるAttentionを可視化します

In [1]:
!git clone https://github.com/YutaroOgawa/pytorch_advanced.git

Cloning into 'pytorch_advanced'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 465 (delta 6), reused 12 (delta 4), pack-reused 441
Receiving objects: 100% (465/465), 14.78 MiB | 5.03 MiB/s, done.
Resolving deltas: 100% (237/237), done.


In [ ]:
cd pytorch_advanced/7_nlp_sentiment_transformer

/content/pytorch_advanced/7_nlp_sentiment_transformer


In [ ]:
ls

7-1_Tokenizer.ipynb                7-7_transformer_training_inference.ipynb
7-2_torchtext.ipynb                data/
7-4_vectorize.ipynb                make_folders_and_data_downloads.ipynb
7-5_IMDb_Dataset_DataLoader.ipynb  utils/
7-6_Transformer.ipynb


In [ ]:
import os
import urllib.request
import zipfile
import tarfile

In [ ]:
# IMDbデータセットをダウンロード。30秒ほどでダウンロードできます

url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
save_path = "./data/aclImdb_v1.tar.gz"
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

In [ ]:
# './data/aclImdb_v1.tar.gz'の解凍　1分ほどかかります

# tarファイルを読み込み
tar = tarfile.open('./data/aclImdb_v1.tar.gz')
tar.extractall('./data/')  # 解凍
tar.close()  # ファイルをクローズ

# フォルダ「data」内にフォルダ「aclImdb」というものができます。

In [ ]:
# fastTextの公式の英語学習済みモデル（650MB）をダウンロード。時間が5分ほどかかります
url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"
save_path = "./data/wiki-news-300d-1M.vec.zip"
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

In [ ]:
# フォルダ「data」内の「/wiki-news-300d-1M.vec.zip」を解凍する

zipf = zipfile.ZipFile("./data/wiki-news-300d-1M.vec.zip")  # zip > zipf
zipf.extractall("./data/")  # ZIPを解凍  # zip > zipf
zipf.close()  # ZIPファイルをクローズ    # zip > zipf

# フォルダ「data」内にフォルダ「wiki-news-300d-1M.vec」というものができます。

# 7.7 学習目標

1.	Transformerの学習を実装できるようになる
2.	Transformerの判定時のAttention可視化を実装できるようになる


# 事前準備

- フォルダ「utils」内の関数やクラスを使用します

In [ ]:
# パッケージのimport
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

import torchtext


In [ ]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# DatasetとDataLoaderを作成

In [ ]:
from utils.dataloader import get_IMDb_DataLoaders_and_TEXT

# 読み込み
train_dl, val_dl, test_dl, TEXT = get_IMDb_DataLoaders_and_TEXT(
    max_length=256, batch_size=64)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dl, "val": val_dl}


100%|█████████▉| 999403/999994 [02:14<00:00, 7807.35it/s]

# ネットワークモデルの作成

In [ ]:
from utils.transformer import TransformerClassification

# モデル構築
net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)

# ネットワークの初期化を定義


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        # Liner層の初期化
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)


# 訓練モードに設定
net.train()

# TransformerBlockモジュールを初期化実行
net.net3_1.apply(weights_init)
net.net3_2.apply(weights_init)


print('ネットワーク設定完了')


100%|█████████▉| 999403/999994 [02:31<00:00, 7807.35it/s]

ネットワーク設定完了


# 損失関数と最適化手法を定義

In [ ]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

# 最適化手法の設定
learning_rate = 2e-5
optimizer = optim.Adam(net.parameters(), lr=learning_rate)


# 学習・検証を実施

In [ ]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')
    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書オブジェクト

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # mask作成
                    input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
                    input_mask = (inputs != input_pad)

                    # Transformerに入力
                    outputs, _, _ = net(inputs, input_mask)
                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net


In [ ]:
# 学習・検証を実行する 15分ほどかかります
num_epochs = 10
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-----start-------
Epoch 1/10 | train |  Loss: 0.5925 Acc: 0.6721
Epoch 1/10 |  val  |  Loss: 0.4293 Acc: 0.8042
Epoch 2/10 | train |  Loss: 0.4434 Acc: 0.7969
Epoch 2/10 |  val  |  Loss: 0.4116 Acc: 0.8140
Epoch 3/10 | train |  Loss: 0.4043 Acc: 0.8184
Epoch 3/10 |  val  |  Loss: 0.3739 Acc: 0.8336
Epoch 4/10 | train |  Loss: 0.3866 Acc: 0.8283
Epoch 4/10 |  val  |  Loss: 0.3674 Acc: 0.8404
Epoch 5/10 | train |  Loss: 0.3711 Acc: 0.8351
Epoch 5/10 |  val  |  Loss: 0.3564 Acc: 0.8468
Epoch 6/10 | train |  Loss: 0.3576 Acc: 0.8413
Epoch 6/10 |  val  |  Loss: 0.3477 Acc: 0.8508
Epoch 7/10 | train |  Loss: 0.3508 Acc: 0.8473
Epoch 7/10 |  val  |  Loss: 0.3719 Acc: 0.8434
Epoch 8/10 | train |  Loss: 0.3434 Acc: 0.8491
Epoch 8/10 |  val  |  Loss: 0.3485 Acc: 0.8558
Epoch 9/10 | train |  Loss: 0.3340 Acc: 0.8572
Epoch 9/10 |  val  |  Loss: 0.3616 Acc: 0.8488
Epoch 10/10 | train |  Loss: 0.3327 Acc: 0.8569
Epoch 10/10 |  val  |  Loss: 0.3369 Acc: 0.8600


# テストデータでの正解率を求める

In [ ]:
# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)

epoch_corrects = 0  # epochの正解数

for batch in (test_dl):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    
    # GPUが使えるならGPUにデータを送る
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # mask作成
        input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
        input_mask = (inputs != input_pad)

        # Transformerに入力
        outputs, _, _ = net_trained(inputs, input_mask)
        _, preds = torch.max(outputs, 1)  # ラベルを予測

        # 結果の計算
        # 正解数の合計を更新
        epoch_corrects += torch.sum(preds == labels.data)

# 正解率
epoch_acc = epoch_corrects.double() / len(test_dl.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(test_dl.dataset),epoch_acc))


テストデータ25000個での正解率：0.8505


# Attentionの可視化で判定根拠を探る



In [ ]:
# HTMLを作成する関数を実装


def highlight(word, attn):
    "Attentionの値が大きいと文字の背景が濃い赤になるhtmlを出力させる関数"

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normlized_weights_1, normlized_weights_2, TEXT):
    "HTMLデータを作成する"

    # indexの結果を抽出
    sentence = batch.Text[0][index]  # 文章
    label = batch.Label[index]  # ラベル
    pred = preds[index]  # 予測

    # indexのAttentionを抽出と規格化
    attens1 = normlized_weights_1[index, 0, :]  # 0番目の<cls>のAttention
    attens1 /= attens1.max()

    attens2 = normlized_weights_2[index, 0, :]  # 0番目の<cls>のAttention
    attens2 /= attens2.max()

    # ラベルと予測結果を文字に置き換え
    if label == 0:
        label_str = "Negative"
    else:
        label_str = "Positive"

    if pred == 0:
        pred_str = "Negative"
    else:
        pred_str = "Positive"

    # 表示用のHTMLを作成する
    html = '正解ラベル：{}<br>推論ラベル：{}<br><br>'.format(label_str, pred_str)

    # 1段目のAttention
    html += '[TransformerBlockの1段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens1):
        html += highlight(TEXT.vocab.itos[word], attn)
    html += "<br><br>"

    # 2段目のAttention
    html += '[TransformerBlockの2段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens2):
        html += highlight(TEXT.vocab.itos[word], attn)

    html += "<br><br>"

    return html


In [ ]:
from IPython.display import HTML

# Transformerで処理

# ミニバッチの用意
batch = next(iter(test_dl))

# GPUが使えるならGPUにデータを送る
inputs = batch.Text[0].to(device)  # 文章
labels = batch.Label.to(device)  # ラベル

# mask作成
input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
input_mask = (inputs != input_pad)

# Transformerに入力
outputs, normlized_weights_1, normlized_weights_2 = net_trained(
    inputs, input_mask)
_, preds = torch.max(outputs, 1)  # ラベルを予測


index = 3  # 出力させたいデータ
html_output = mk_html(index, batch, preds, normlized_weights_1,
                      normlized_weights_2, TEXT)  # HTML作成
HTML(html_output)  # HTML形式で出力


In [ ]:
index = 61  # 出力させたいデータ
html_output = mk_html(index, batch, preds, normlized_weights_1,
                      normlized_weights_2, TEXT)  # HTML作成
HTML(html_output)  # HTML形式で出力


以上